# Part 1: Importing and cleaning the data

In [1]:
import pandas as pd
from ast import literal_eval

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import folium
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
# The CSV was given in another Coursera course, so I will just import it as a CSV

schools = pd.read_csv('schools.csv')

#Turn the string location tuples into a list of Python tuples

location_tuples = schools['Location'].apply(lambda location: literal_eval(location))

# Add location tuples to main dataframe

schools['Latitude'] = location_tuples.apply(lambda t: t[0])
schools['Longitude'] = location_tuples.apply(lambda t: t[1])

"""# Take only the columns we really need

columns = ['NAME_OF_SCHOOL', 'Latitude', 'Longitude']
schools = schools[columns]"""
schools.rename(columns={"NAME_OF_SCHOOL": "School"}, inplace=True)

schools.head()

,School ID,School,"Elementary, Middle, or High School",Street Address,City,State,ZIP Code,Phone Number,Link,Network Manager,...,Freshman on Track Rate %,X_COORDINATE,Y_COORDINATE,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,Ward,Police District,Location
0,610038,Abraham Lincoln Elementary School,ES,615 W Kemper Pl,Chicago,IL,60614,(773) 534-5720,http://schoolreports.cps.edu/SchoolProgressRep...,Fullerton Elementary Network,...,NDA,1171699.458,1915829.428,41.924497,-87.644522,7,LINCOLN PARK,43,18,"(41.92449696, -87.64452163)"
1,610281,Adam Clayton Powell Paideia Community Academy ...,ES,7511 S South Shore Dr,Chicago,IL,60649,(773) 535-6650,http://schoolreports.cps.edu/SchoolProgressRep...,Skyway Elementary Network,...,NDA,1196129.985,1856209.466,41.760324,-87.556736,43,SOUTH SHORE,7,4,"(41.76032435, -87.55673627)"
2,610185,Adlai E Stevenson Elementary School,ES,8010 S Kostner Ave,Chicago,IL,60652,(773) 535-2280,http://schoolreports.cps.edu/SchoolProgressRep...,Midway Elementary Network,...,NDA,1148427.165,1851012.215,41.747111,-87.731702,70,ASHBURN,13,8,"(41.74711093, -87.73170248)"
3,609993,Agustin Lara Elementary Academy,ES,4619 S Wolcott Ave,Chicago,IL,60609,(773) 535-4389,http://schoolreports.cps.edu/SchoolProgressRep...,Pershing Elementary Network,...,NDA,1164504.290,1873959.199,41.809757,-87.672145,61,NEW CITY,20,9,"(41.8097569, -87.6721446)"
4,610513,Air Force Academy High School,HS,3630 S Wells St,Chicago,IL,60609,(773) 535-1590,http://schoolreports.cps.edu/SchoolProgressRep...,Southwest Side High School Network,...,91.8,1175177.622,1880745.126,41.828146,-87.632794,34,ARMOUR SQUARE,11,9,"(41.82814609, -87.63279369)"


**Now we're going to add the Foursquare data and get a list of venues nearby each school**

In [3]:
# Set up url parameters

CLIENT_ID = '0V3TLIC2XB0C0WRJW5YRZXJBQEUACTOWUWUNV2N3MZZQJCC1'
CLIENT_SECRET = 'ZLHT4HW1R4IQRPBOUR3GXLLVULWXPHMB20YESYRQZ2VITVV2'
VERSION = '20180605'
radius = 500
LIMIT = 100

In [4]:
# Define the nearby venues function

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['School', 
                  'School Latitude', 
                  'School Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [5]:
# This was the code used to get the venue data, but to avoid killing their API whenever we run this script,
# the data has been pickled. The pickle will be included in the Github Repository.

"""school_venues = getNearbyVenues(names=schools['School Name'],
                                   latitudes=schools['Latitude'],
                                   longitudes=schools['Longitude']
                                  )"""

"school_venues = getNearbyVenues(names=schools['School Name'],\n                                   latitudes=schools['Latitude'],\n                                   longitudes=schools['Longitude']\n                                  )"

In [6]:
school_venues = pd.read_pickle('./school_venues.pkl')
school_venues.head()

,School,School Latitude,School Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abraham Lincoln Elementary School,41.924497,-87.644522,Swirlz Cupcakes,41.923668,-87.646759,Cupcake Shop
1,Abraham Lincoln Elementary School,41.924497,-87.644522,Insomnia Cookies,41.923177,-87.645636,Dessert Shop
2,Abraham Lincoln Elementary School,41.924497,-87.644522,Rickshaw Republic,41.924123,-87.646898,Indonesian Restaurant
3,Abraham Lincoln Elementary School,41.924497,-87.644522,Philz Coffee,41.924879,-87.647094,Coffee Shop
4,Abraham Lincoln Elementary School,41.924497,-87.644522,Potbelly Sandwich Shop,41.923126,-87.645914,Sandwich Place


In [7]:
school_venues.shape

(10030, 7)

In [8]:
venues_onehot = pd.get_dummies(school_venues["Venue Category"], prefix="", prefix_sep="")
venues_onehot.head()

,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,...,Vineyard,Warehouse,Warehouse Store,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# Add the school name in

venues_onehot['School'] = school_venues['School']
n = venues_onehot.columns.get_loc("School")
columns = [venues_onehot.columns[n]] + list(venues_onehot.columns[:n]) + list(venues_onehot.columns[n+1:])
venues_onehot = venues_onehot[columns]
venues_onehot.head()

,School,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Antique Shop,Arcade,...,Vineyard,Warehouse,Warehouse Store,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Abraham Lincoln Elementary School,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Abraham Lincoln Elementary School,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Abraham Lincoln Elementary School,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Abraham Lincoln Elementary School,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Abraham Lincoln Elementary School,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Now we're going to group all of these together and take the mean, giving us a one-row snapshot of the general distribution of venue near each school**

In [10]:
schools_grouped = venues_onehot.groupby('School').mean().reset_index()
schools_grouped.head()

,School,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Antique Shop,Arcade,...,Vineyard,Warehouse,Warehouse Store,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,A.N. Pritzker School,0.0,0.03,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.00,0.0,0.01
1,Abraham Lincoln Elementary School,0.0,0.00,0.0,0.0,0.0,0.0,0.01,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.01,0.0,0.00
2,Adam Clayton Powell Paideia Community Academy ...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00
3,Adlai E Stevenson Elementary School,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00
4,Agustin Lara Elementary Academy,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00


By this point we have a list of all the schools, combined with the mean score for each venue type. We can use this data to run our kmeans clustering algorithm and segment these schools into categories.

In [11]:
# set number of clusters
kclusters = 5

schools_grouped_clustering = schools_grouped.drop('School', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=24).fit(schools_grouped_clustering)

In [12]:
# Insert the label data into the schools dataframe

schools_grouped.insert(1, 'Cluster Label', kmeans.labels_)

In [13]:
school_labels = schools_grouped[['School', 'Cluster Label']]

In [14]:
# Join the labels with our original schools dataset so we can have some fun analyzing the data

schools_labeled = schools.join(school_labels.set_index('School'), on="School")

# Remove schools without cluster labels

mask = pd.isna(schools_labeled['Cluster Label'])
schools_labeled = schools_labeled.dropna()

# Cast cluster label as int

schools_labeled['Cluster Label'] = schools_labeled['Cluster Label'].astype("int32")


In [15]:
schools_labeled["Cluster Label"].value_counts()

3    271
1     62
2     50
0     47
Name: Cluster Label, dtype: int64

In [16]:
import random

latitude = schools_labeled['Latitude'].mean()
longitude = schools_labeled['Longitude'].mean()

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
#x = np.arange(kclusters)
#ys = [i + x + (i*x)**2 for i in range(kclusters)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]
rainbow = ['#ff1100', '#22a4f5', '#088210', '#a500ab', '#f5f122']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(schools_labeled['Latitude'], schools_labeled['Longitude'], schools_labeled['School'], schools_labeled['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Let's see if we can figure out what different venues characterize the different clusters**

In [17]:
# Group the schools by cluster label and get the mean value of different venue types

schools_clustered = schools_grouped.groupby('Cluster Label').mean().reset_index()

In [18]:
# Now we just sort the columns by value

venue_columns = schools_clustered.columns[2:]
venue_scores = schools_clustered[venue_columns]
venue_scores

,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,...,Vineyard,Warehouse,Warehouse Store,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0.000215,0.00000,0.000000,0.000000,0.000000,0.005337,0.000410,0.000000,0.000614,0.003068,...,0.000000,0.000000,0.00000,0.000000,0.001453,0.002703,0.000429,0.002206,0.000000,0.000215
1,0.000612,0.00000,0.002012,0.000000,0.000000,0.027904,0.000000,0.000000,0.000000,0.001083,...,0.000000,0.000000,0.00000,0.000782,0.000000,0.004695,0.000000,0.000587,0.001657,0.000000
2,0.000000,0.00000,0.000000,0.002604,0.002604,0.016146,0.000000,0.002604,0.000000,0.005357,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.007031,0.000000,0.001202,0.001953,0.001202
3,0.001179,0.00009,0.000077,0.002984,0.000476,0.025886,0.001406,0.000234,0.000381,0.007790,...,0.000119,0.001508,0.00015,0.001314,0.000406,0.003910,0.001564,0.003490,0.002573,0.002254
4,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [19]:
top_venues = []

for index, row in venue_scores.iterrows():
    top_venues.append(row.sort_values(ascending=False).index.values.tolist()[:10])

In [20]:
top_venues_df = pd.DataFrame.from_records(top_venues)
top_venues_df.head()

,0,1,2,3,4,5,6,7,8,9
0,Mexican Restaurant,Pizza Place,Park,Bakery,Sandwich Place,Grocery Store,Taco Place,Donut Shop,Pharmacy,Fast Food Restaurant
1,Fast Food Restaurant,Fried Chicken Joint,Discount Store,Train Station,Liquor Store,Park,Intersection,American Restaurant,Sandwich Place,Currency Exchange
2,Park,Fast Food Restaurant,Intersection,Bus Station,Train Station,Gym,Pizza Place,Liquor Store,Grocery Store,American Restaurant
3,Pizza Place,Sandwich Place,Bar,American Restaurant,Park,Mexican Restaurant,Grocery Store,Bus Station,Chinese Restaurant,Coffee Shop
4,Construction & Landscaping,Yoga Studio,Floating Market,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market


In [21]:
cluster_venue_rankings = pd.concat([schools_clustered["Cluster Label"], top_venues_df], axis=1)
cluster_venue_rankings.head()

,Cluster Label,0,1,2,3,4,5,6,7,8,9
0,0,Mexican Restaurant,Pizza Place,Park,Bakery,Sandwich Place,Grocery Store,Taco Place,Donut Shop,Pharmacy,Fast Food Restaurant
1,1,Fast Food Restaurant,Fried Chicken Joint,Discount Store,Train Station,Liquor Store,Park,Intersection,American Restaurant,Sandwich Place,Currency Exchange
2,2,Park,Fast Food Restaurant,Intersection,Bus Station,Train Station,Gym,Pizza Place,Liquor Store,Grocery Store,American Restaurant
3,3,Pizza Place,Sandwich Place,Bar,American Restaurant,Park,Mexican Restaurant,Grocery Store,Bus Station,Chinese Restaurant,Coffee Shop
4,4,Construction & Landscaping,Yoga Studio,Floating Market,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market


Conclusions about what these venue frequncies mean for the various distributions can best be left up to policy makers.

Additional research could try to characterize the various clusters by comparing cluster labels to school performance data.

Here we will perform a very preliminry look at this by examining cluster means for a select few metrics.

In [22]:
metric_columns = [
    'School',
    'Cluster Label',
    'Instruction Score', 
    'Teachers Score', 
    'AVERAGE_STUDENT_ATTENDANCE', 
    'Rate of Misconducts (per 100 students) ']

metric_df = schools_labeled[metric_columns]

mask = (metric_df["Teachers Score"] != "NDA")
metric_df = metric_df[mask]

mask = (metric_df["AVERAGE_STUDENT_ATTENDANCE"] != "NDA")
metric_df = metric_df[mask]

metric_df["Teachers Score"] = metric_df["Teachers Score"].astype("int32")
metric_df["AVERAGE_STUDENT_ATTENDANCE"] = metric_df["AVERAGE_STUDENT_ATTENDANCE"].map(lambda x: x.rstrip('%'))
metric_df["AVERAGE_STUDENT_ATTENDANCE"] = metric_df["AVERAGE_STUDENT_ATTENDANCE"].astype("float64")

In [23]:
metrics = metric_df.groupby("Cluster Label").mean().reset_index()

In [24]:
metrics

,Cluster Label,Instruction Score,Teachers Score,AVERAGE_STUDENT_ATTENDANCE,Rate of Misconducts (per 100 students)
0,0,44.642857,43.785714,95.878571,12.275000
1,1,53.090909,46.969697,93.357576,29.254545
2,2,52.368421,46.789474,94.278947,22.863158
3,3,52.331081,51.601351,94.675676,17.150676


So it looks like it is very hard to segment these schools based on nearby venues. The only thing that shows any clear separation is rate of misconducts. If I were giving a suggestion to educational policymakers, I would suggest looking into the relationship between misconduct and location of school.